# Práctica 1. Resolución de problemas con búsqueda 
## Inteligencia Artificial I    2022/2023

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

**Fecha de entrega de la práctica completa: 18 de octubre** 

### Miembros del grupo 2
 - Fernando Leal
 - Pablo Martín
 - Jinqing Cai

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en representar estados y acciones, la comprobación de objetivo y el coste de las acciones.
La siguiente clase Problem (de la librería AIMA) representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problem, y requerirá implementar
* estado_inicial, es_estado_final(_), acciones(_), aplica(_,_), coste_de_aplicar_accion y eventualmente __init__ 

El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 

 __Lo más sencillo para empezar es copiar en tu directorio de trabajo los archivos _search.py y utils.py_ de AIMA__

In [1]:
from search import Problem

## Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.


In [2]:
class Problem(object):
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo __definir un problema como subclase
de problema__. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [5]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0, 0)

    def actions(self, estado):
        jarra_de_4 = estado[0]
        jarra_de_3 = estado[1]
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs = list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self, estado, accion):
        # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        j4 = estado[0]
        j3 = estado[1]
        if accion == "llenar jarra de 4":
            return (4, j3)
        elif accion == "llenar jarra de 3":
            return (j4, 3)
        elif accion == "vaciar jarra de 4":
            return (0, j3)
        elif accion == "vaciar jarra de 3":
            return (j4, 0)
        elif accion == "trasvasar de jarra de 4 a jarra de 3":
            return (j4 - 3 + j3, 3) if j3 + j4 >= 3 else (0, j3 + j4)
        else:  #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3 + j4, 0) if j3 + j4 <= 4 else (4, j3 - 4 + j4)

    def goal_test(self, estado):
        return estado[0] == 2

Vamos a probar algunos ejemplos.

In [6]:
# Estado inicial
p = Jarras()
p.initial

(0, 0)

In [7]:
# Acciones 
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [8]:
p.result(p.initial, "llenar jarra de 4")

(4, 0)

In [9]:
# p.coste_de_aplicar_accion(p.initial, "llenar jarra de 4")

In [10]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [11]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Aunque no es necesario puedes hacer tu propia implementación del problema 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''

    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        misionerosIzq = s[0]
        canibalesIzq = s[1]
        return (misionerosIzq >= canibalesIzq or misionerosIzq == 0) and ((3 - misionerosIzq) >= (3 - canibalesIzq) or misionerosIzq == 3) and ((0 <= misionerosIzq) and (misionerosIzq <= 3)) and ((0 <= canibalesIzq) and (canibalesIzq <= 3))

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)

    
    ## Heuristicas del problema
    def h1(self, s):
        misionerosIzq = s.state[0]
        canibalesIzq = s.state[1]

        # Si no he entendido mal es esta la heuristica que hay que tomar
        return (misionerosIzq + canibalesIzq) / 2
    
    def h2(self, s):
        misionerosIzq = s.state[0]
        canibalesIzq = s.state[1]
        lado = 1 - s.state[2]
        
        # En la formula de la diapositiva, hay un parametro orilla que vale 1 si barca esta en izquierda, 
        #   y 0 si barca esta en orilla final. En nuestra representacion, lado == 0 entonces la barca esta en izquierda
        #   y 1 si esta en otro lado => Hay que invertirlo
        return 2 * (canibalesIzq + misionerosIzq) - lado

In [12]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

In [13]:
misioneros.initial

(3, 3, 0)

In [14]:
# Acciones aplicables en el estado inicial
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

Por ejemplo, para resolver el problema de los misioneros con el método de busqueda en anchura la llamada sería:  

```python
from search import *
breadth_first_tree_search(misioneros).solution()
```

In [15]:
from search import *
breadth_first_tree_search(misioneros).solution()
# Tiene pinta de que va bien

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

### Representación del problema del puzzle de 8

> Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [16]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        pos_hueco = estado.index(0)  # busco la posicion del 0
        accs = list()
        if pos_hueco not in (0, 1, 2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0, 3, 6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (6, 7, 8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (2, 5, 8):
            accs.append("Mover hueco derecha")

        return accs

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        def swap(x):
            l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]
        if accion == "Mover hueco arriba":
            swap(-3)
        elif accion == "Mover hueco abajo":
            swap(3)
        elif accion == "Mover hueco izquierda":
            swap(-1)
        elif accion == "Mover hueco derecha":
            swap(1)

        return tuple(l)

    def h(self, node):
        """ Return the heuristic value for a given state."""
        return 1
    
    # Hemos añadido las heurísticas a la clase para poder acceder al atributo goal
    def linear(self, node):
        #goal = node.goal
        return sum(x != sol for x, sol in zip(node.state, self.goal))

    def manhattan(self, node):
        state = node.state
        mhd = sum(abs(x-y) for x,y in zip(state, self.goal))
        return mhd

    def sqrt_manhattan(self, node):
        import math
        mhd = self.manhattan(node)
        return math.sqrt(mhd)

    def max_heuristic(self, node):
        score1 = self.manhattan(node)
        score2 = self.linear(node)
        return max(score1, score2)

    def coste_de_aplicar_accion(self, s, a):
        return 1
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0  

In [17]:
def show(s):
    print(" ".join(map(str,s[:3])))
    print(" ".join(map(str,s[3:6])))
    print(" ".join(map(str,s[6:])))

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [18]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
show(p8.initial)

2 8 3
1 6 4
7 0 5


In [20]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [21]:
show(p8.result(p8.initial, "Mover hueco arriba"))

2 8 3
1 0 4
7 6 5


In [22]:
# show(p8.result(p8.initial, "Mover hueco abajo"))

# Da error porque estamos realizando una accion invalida, 
# no presente en la lista de acciones permitidas. Además, viola la capacidad del array, por eso lanza el error
# En el código no estamos comprobando si una acción es válida o no al hacer .result

In [23]:
show(p8.result(p8.initial, "Mover hueco derecha"))

2 8 3
1 6 4
7 5 0


In [24]:
p8.coste_de_aplicar_accion(p8.initial, "Mover hueco abajo")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzzle de 16 piezas.

In [25]:
class Quince_Puzzle(Problem):
    ## Completa la definición del problema
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        self.side = 4
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        pos_hueco = estado.index(0)  # busco la posicion del 0
        accs = list()
        if pos_hueco not in (0, 1, 2, 3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0, 4, 8, 12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (12, 13, 14, 15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (3, 7, 11, 15):
            accs.append("Mover hueco derecha")
            
        return accs

    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # La condicion de si es resoluble de 15 puzzle es más complicado:
        # https://www.geeksforgeeks.org/check-instance-15-puzzle-solvable/
        # https://puzzling.stackexchange.com/a/54905
        # Al tener tablero de longitud par (N = 4), no basta con devolver la partidad de inversiones

        inversion = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1

        # Hallar la posicion del hueco (entre 0 y 15)
        # Necesito hallar la fila en la que se encuentra el hueco, contando desde abajo (indexado por 1)
        # Se puede hallar la fila desde arriba haciendo division entera (//):
        pos_hueco = state.index(0)
        fila = pos_hueco // 4
        fila_abajo = 4 - fila

        # Si fila_abajo es par => resoluble si el numero de inversiones es impar
        # Si fila_abajo es impar => resoluble si el numero de inversiones es par
        # print(fila_abajo)

        if fila_abajo % 2 == 1:
            return inversion % 2 == 0
        else:
            return inversion % 2 == 1
    
    #Heurísticas que hemos implementado
    def linear(self, node):
        #goal = node.goal
        return sum(x != sol for x, sol in zip(node.state, self.goal))

    def manhattan(self, node):
        state = node.state
        mhd = sum(abs(x-y) for x,y in zip(state, self.goal))
        return mhd

    def sqrt_manhattan(self, node):
        import math
        mhd = self.manhattan(node)
        return math.sqrt(mhd)

    def max_heuristic(self, node):
        score1 = self.manhattan(node)
        score2 = self.linear(node)
        return max(score1, score2)

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        def swap(x):
            l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]
        if accion == "Mover hueco arriba":
            swap(-self.side)
        elif accion == "Mover hueco abajo":
            swap(self.side)
        elif accion == "Mover hueco izquierda":
            swap(-1)
        elif accion == "Mover hueco derecha":
            swap(1)

        return tuple(l)

    def h(self, node):
        """ Return the heuristic value for a given state."""
        return 1
    
    def coste_de_aplicar_accion(self, s, a):
        return 1

In [26]:
# Para ver que esta bien definida la funcion de resolubilidad
test_15p = Quince_Puzzle((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 13, 14, 15, 12))
test_15p.check_solvability(test_15p.initial) == True

True

## Algunas para ver que la clase de `Quince_Puzzle` está bien definida

In [27]:
def show_15puzzle(estado):
    top_line    = "┌"+"┬".join(["──"]*4)+"┐\n"
    split_line  = "├"+"┼".join(["──"]*4)+"┤\n"
    bottom_line = "└"+"┴".join(["──"]*4)+"┘\n"
    rows = ("│"+"│".join(map(lambda x : (str(x) if x != 0 else ' ').rjust(2," "),estado[4*i:4*(i+1)]))+"│\n" for i in range(4))
    print(top_line + split_line.join(rows) + bottom_line)

In [28]:
p15 = Quince_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5, 9, 10, 11, 12, 14, 13, 15))
show_15puzzle(p15.initial)

┌──┬──┬──┬──┐
│ 2│ 8│ 3│ 1│
├──┼──┼──┼──┤
│ 6│ 4│ 7│  │
├──┼──┼──┼──┤
│ 5│ 9│10│11│
├──┼──┼──┼──┤
│12│14│13│15│
└──┴──┴──┴──┘



In [29]:
p15.actions(p15.initial)

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo']

In [30]:
show_15puzzle(p15.result(p15.initial, "Mover hueco arriba"))

┌──┬──┬──┬──┐
│ 2│ 8│ 3│  │
├──┼──┼──┼──┤
│ 6│ 4│ 7│ 1│
├──┼──┼──┼──┤
│ 5│ 9│10│11│
├──┼──┼──┼──┤
│12│14│13│15│
└──┴──┴──┴──┘



In [31]:
# No es un movimiento válido, pero asumimos que .result recibe una acción válida
show_15puzzle(p15.result(p15.initial, "Mover hueco derecha"))

┌──┬──┬──┬──┐
│ 2│ 8│ 3│ 1│
├──┼──┼──┼──┤
│ 6│ 4│ 7│ 5│
├──┼──┼──┼──┤
│  │ 9│10│11│
├──┼──┼──┼──┤
│12│14│13│15│
└──┴──┴──┴──┘



In [32]:
p15.coste_de_aplicar_accion(p15.initial, "Mover hueco abajo")

1

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [33]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [34]:
# # una opcion para poner un prefijo a la ruta antes de importar es:
# import sys
# sys.path.append('aima-python/')
# import search

In [35]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [36]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [37]:
# Ejemplo de salida:
# ['llenar jarra de 3',
#  'trasvasar de jarra de 3 a jarra de 4',
#  'llenar jarra de 3',
#  'trasvasar de jarra de 3 a jarra de 4',
#  'vaciar jarra de 4',
#  'trasvasar de jarra de 3 a jarra de 4']

In [38]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

#### Problema misioneros

In [39]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [40]:
depth_first_graph_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

#### Problema del puzzle del 8

In [41]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
show(p8.initial)

2 8 3
1 6 4
7 0 5


In [42]:
show(p8.goal)

1 2 3
4 5 6
7 8 0


In [43]:
# breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si es método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## ...............................................................................

Es cierto que BFS es completo, sin embargo, **este estado inicial no tiene solución** (lo comprobamos a continuación). Y se metería en bucle infinito porque no guardamos estados pasados para hacer control de repetidos, y entonces se mete en estados que hemos visitado ya anteriormente (haciendo acciones inversas).

In [44]:
# # Añade a la definición del problema Ocho_Puzzle la función solvability que comprueba si un estado inicial tiene solución.
# def check_solvability(self, state):
#         """ Checks if the given state is solvable """
# # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
# #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
#         inversion = 0
#         for i in range(len(state)):
#             for j in range(i+1, len(state)):
#                 if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
#                     inversion += 1
#         return inversion % 2 == 0        

In [45]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol = breadth_first_tree_search(p8).solution()
else:
    sol = "Este estado inicial no tiene solucion"

In [46]:
sol

'Este estado inicial no tiene solucion'

In [47]:
show((1, 2, 3, 4, 5, 6, 0, 7, 8))
breadth_first_tree_search(Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))).solution()

1 2 3
4 5 6
0 7 8


['Mover hueco derecha', 'Mover hueco derecha']

In [48]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [49]:

moves = breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']
s = (2, 4, 3, 1, 5, 6, 7, 8, 0)
show((2, 4, 3, 1, 5, 6, 7, 8, 0))
for action in moves:
    s = estado.result(s,action)
    print()
    show(s)

2 4 3
1 5 6
7 8 0

2 4 3
1 5 0
7 8 6

2 4 3
1 0 5
7 8 6

2 0 3
1 4 5
7 8 6

0 2 3
1 4 5
7 8 6

1 2 3
0 4 5
7 8 6

1 2 3
4 0 5
7 8 6

1 2 3
4 5 0
7 8 6

1 2 3
4 5 6
7 8 0


#### Problema del puzzle de 15

In [91]:
# El problema tiene solución, pero tardará mucho en encontrarlo al ser de tipo DFS
# Tiene que encontrarlo, porque es un graph_search (con control de repetidos), y el problema tiene solución
depth_first_graph_search(Quince_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5, 9, 10, 11, 12, 14, 13, 15))).solution()

breadth_first_graph_search(Quince_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5, 9, 10, 11, 12, 14, 13, 15))).solution()

KeyboardInterrupt: KeyboardInterrupt: 

In [92]:
# Sin control de repetidos -> Puede meterse en bucle infinito
# breadth_first_tree_search(Quince_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5, 9, 10, 11, 12, 14, 13, 15))).solution()

### Comentarios del ejercicio 3. 
#### Se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:

- **Misioneros**: al ser un problema simple y que no tiene gran variedad de soluciones el algoritmo termina rápidamente y nos proporciona una solución acertada. La búsqueda en anchura tarda ligeramente menos.
- **Puzzle de 8**: en el primer caso ocurre un bucle infinito, al no llevar un recuento de los estados repetidos. Entramos en un ciclo del que el programa no es capaz de escapar. Una vez corregido el programa nos dice que no se puede resolver el problema. En el segundo caso.
- **Puzzle de 15**: en ambos casos la búsqueda tarda demasiado y al final nos quedamos sin memoria en el sistema.

### EJERCICIO 4:  Definición de heurísticas

#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
Puedes utilizar alguna de las heurísticas vistas en clase.

In [51]:
# Heurísticas para el problema de los misioneros.
def heur_misioneros_1(c, n, b):
    return (c + n) / 2
def heur_misioneros_2(c, n, b):
    return 2 * (c + n) - b
# (n_misioneros, n_canibales, barca_donde (0 si esta en lado final, 1 en lado izquierdo))

1. La primera heuristica consiste en asumir en que hay infinitas barcas en ambos lados de la orilla. En este caso, la solución sería hacer que viaje un caníbal y un misionero juntos siempre (entre los que quedan en la orilla de izquierda).
2. La segunda heurística consiste en quitar la restricción de que los caníbales comen a los misioneros, y hacer que en cada viaje transportamos 2 personas, y luego vuelve uno. Por lo tanto necesitaría `2 * num_personas` viajes. La resta del final es para que, en el caso de que el barco estaba inicialmente en el lado izquierdo, no hace falta hacer el viaje de ida al final.

#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas:
* **linear(node)**: cuenta el número de casillas mal colocadas respecto al estado final. 
* **manhattan(node)**: suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final. 
* **max_heuristic(node)**: maximo de las dos anteriores 
* **sqrt_manhattan(node)**: raíz cuadrada de la distancia Manhattan

In [52]:
# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 

def linear(node):
    #goal = node.state.goal
    return sum(x != sol for x, sol in zip(node.state, node.goal))

def manhattan(node):
    state = node.state
    mhd = sum(abs(x-y) for x,y in zip(state, node.goal))
    return mhd

def sqrt_manhattan(node):
    import math
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

#### Problema de los misioneros

In [53]:
%timeit uniform_cost_search(misioneros)
%timeit best_first_graph_search(misioneros,misioneros.h1)
%timeit best_first_graph_search(misioneros,misioneros.h2)
%timeit astar_search(misioneros,misioneros.h1)
%timeit astar_search(misioneros,misioneros.h2)

225 µs ± 787 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
218 µs ± 6.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
218 µs ± 6.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
235 µs ± 563 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
241 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Observamos que ambas métricas funcionan de forma similar. `best_first_graph_search` da el mejor resultado, pues implementa un "Dijkstra dirigido" que nos permite capitalizar en la buena elección de las heurísticas para llegar rápidamente al resultado. `astar_search` usa las mismas heurísticas, pero por el hecho de combinarlas con el coste del camino hasta cada nodo, hace que dependa menos de ellas, lo cual perjudica al problema en esta situación.

#### Problema del puzzle de 8

In [54]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [55]:
from search import astar_search
print(puzzle.goal)
astar_search(puzzle, puzzle.manhattan).solution()

(1, 2, 3, 4, 5, 6, 7, 8, 0)


['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [56]:
puzzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [57]:
puzzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [58]:
astar_search(puzzle, puzzle.linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [59]:
astar_search(puzzle, puzzle.max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [60]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [61]:
%%timeit
astar_search(puzzle_1, puzzle_1.linear)
astar_search(puzzle_2, puzzle_2.linear)
astar_search(puzzle_3, puzzle_3.linear)

4.74 ms ± 72.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%%timeit
astar_search(puzzle_1, puzzle_1.manhattan)
astar_search(puzzle_2, puzzle_2.manhattan)
astar_search(puzzle_3, puzzle_3.manhattan)

10.7 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%%timeit
astar_search(puzzle_1, puzzle_1.sqrt_manhattan)
astar_search(puzzle_2, puzzle_2.sqrt_manhattan)
astar_search(puzzle_3, puzzle_3.sqrt_manhattan)

151 ms ± 910 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%%timeit
astar_search(puzzle_1, puzzle_1.max_heuristic)
astar_search(puzzle_2, puzzle_2.max_heuristic)
astar_search(puzzle_3, puzzle_3.max_heuristic)

11.5 ms ± 40.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



| Heurística | Tiempo medio |
|--|--|
|Lineal | 4.68 ms |
|Manhattan| 10.5 ms |
|Raíz cuadrada de manhattan  | 149 ms |
|Máximo | 16.6 ms |
---

Observamos que el mejor resultado proviene de usar la heurística lineal en cuanto a velocidad de ejecución. Aunque esta sea una heurística muy simple, nos proporciona la suficiente información para saber si nos acercamos o alejamos de la solución real.

#### 15 Puzzles + Heurísticas

####  Realiza pruebas también con el puzzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.

Esta cuestión la respondemos a continuación

In [65]:
estado = (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0)
show_15puzzle(estado)

┌──┬──┬──┬──┐
│ 1│ 2│ 3│ 4│
├──┼──┼──┼──┤
│ 5│ 6│ 7│ 8│
├──┼──┼──┼──┤
│ 9│10│12│15│
├──┼──┼──┼──┤
│13│14│11│  │
└──┴──┴──┴──┘



In [66]:

puzzle = Quince_Puzzle(estado)
astar_search(puzzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [67]:
#Heurística linear
%timeit astar_search(puzzle, puzzle.linear)
#Heurística de Manhattan
%timeit astar_search(puzzle, puzzle.manhattan)
#Heurística square Manhattan
%timeit astar_search(puzzle, puzzle.sqrt_manhattan)
#Heurística Máxima
%timeit astar_search(puzzle, puzzle.max_heuristic)

139 µs ± 437 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.2 ms ± 7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
666 µs ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.43 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Estos son los resultados obtenidos de las heurísticas para el problema dado

| Heurística | Tiempo medio |
|--|--|
|Linear | 134 µs |
| Manhattan | 1170 µs |
|Raíz cuadrada de manhattan | 637 µs |
|Máximo | 1360 µs |
---

Si los comparamos con los obtenidos para el puzzle del ocho, observamos que la heurística Manhattan tiene un rendimiento notablemente peor en el caso escogido, y la raíz cuadrada de la distancia Manhattan, aunque no llega a superar a la heurística linea, sí que muestra un buen rendimiento

---


| Heurística | Tiempo medio |
|--|--|
|Lineal | 4.68 ms |
|Máximo | 16.6 ms |
|Raíz cuadrada de manhattan  | 149 ms |
|Manhattan| 10.5 ms |
---
Esto sin embargo podría ser simplemente un resultado de tener un estado inicial simple, es decir, que exista un camino a la solución en un número reducido de pasos. Por eso, procedemos a hacer un test en casos más generales

In [68]:
# Definimos una función para generar estados aleatorios del tablero
def gen_random_state(depth= 5):
    from random import choice
    state = tuple(list(range(1,16))+[0])
    p = Quince_Puzzle(state)

    for _ in range(depth):
        state = p.result(state,choice(p.actions(state)))
    return state

In [73]:
problemas = [Quince_Puzzle(gen_random_state(i)) for i in range(3,8)]

for i,prob in enumerate(problemas):
    print(f"Puzzle {i+1}")
    show_15puzzle(prob.initial)
def test(heur, problems = problemas):
    for puzzle in problems:
        if heur == "linear": 
            astar_search(puzzle, puzzle.linear)
        elif heur == "manhattan":
            astar_search(puzzle, puzzle.manhattan)
        elif heur == "sqrt_manhattan":
            astar_search(puzzle, puzzle.sqrt_manhattan)
        elif heur == "max_heuristic": 
            astar_search(puzzle,puzzle.max_heuristic)
        else:
            raise ValueError("Unrecognized heuristic")

Puzzle 1
┌──┬──┬──┬──┐
│ 1│ 2│ 3│ 4│
├──┼──┼──┼──┤
│ 5│ 6│ 7│ 8│
├──┼──┼──┼──┤
│ 9│10│11│12│
├──┼──┼──┼──┤
│13│14│  │15│
└──┴──┴──┴──┘

Puzzle 2
┌──┬──┬──┬──┐
│ 1│ 2│ 3│ 4│
├──┼──┼──┼──┤
│ 5│ 6│ 7│ 8│
├──┼──┼──┼──┤
│ 9│10│15│11│
├──┼──┼──┼──┤
│13│  │14│12│
└──┴──┴──┴──┘

Puzzle 3
┌──┬──┬──┬──┐
│ 1│ 2│ 3│ 4│
├──┼──┼──┼──┤
│ 5│ 6│ 7│ 8│
├──┼──┼──┼──┤
│ 9│10│11│  │
├──┼──┼──┼──┤
│13│14│15│12│
└──┴──┴──┴──┘

Puzzle 4
┌──┬──┬──┬──┐
│ 1│ 2│  │ 4│
├──┼──┼──┼──┤
│ 5│ 6│ 3│ 8│
├──┼──┼──┼──┤
│ 9│10│ 7│11│
├──┼──┼──┼──┤
│13│14│15│12│
└──┴──┴──┴──┘

Puzzle 5
┌──┬──┬──┬──┐
│ 1│ 2│ 3│ 4│
├──┼──┼──┼──┤
│ 5│ 6│ 7│ 8│
├──┼──┼──┼──┤
│ 9│  │15│11│
├──┼──┼──┼──┤
│13│10│14│12│
└──┴──┴──┴──┘



In [70]:
%timeit test("linear")
%timeit test("manhattan")
%timeit test("sqrt_manhattan")
%timeit test("max_heuristic")

547 µs ± 9.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.26 ms ± 4.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.62 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.54 ms ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Como podemos observar, los tiempos que nos quedan muestran que la heurística de la raíz cuadrada de la distancia Manhattan empeora cuanto más "desordenado" esté el estado inicial

Usemos un caso mucho más complejo para comprobar que esto sigue siendo verdad

In [87]:
complicado = Quince_Puzzle(gen_random_state(depth=50))
show_15puzzle(complicado.initial)

┌──┬──┬──┬──┐
│ 1│ 3│ 4│11│
├──┼──┼──┼──┤
│ 5│ 2│ 6│ 7│
├──┼──┼──┼──┤
│ 9│10│15│ 8│
├──┼──┼──┼──┤
│13│  │14│12│
└──┴──┴──┴──┘



In [88]:
%timeit test("linear",         problems=[complicado])
%timeit test("manhattan",      problems=[complicado])
%timeit test("sqrt_manhattan", problems=[complicado])
%timeit test("max_heuristic",  problems=[complicado])

52.2 ms ± 364 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.69 ms ± 86.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5min 46s ± 42.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.28 ms ± 34.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Como podemos observar, parece ser que la heurística de la raíz cuadrada de Manhattan tiene mayores complicaciones para encontrar la solución cuanto más desordenado está el estado inicial.

Esto tiene sentido pues `sqrt_manhattan` es menos informada que `manhattan`, pues al tratar de posiciones enteras, las distancias entre fichas siempre son valores mayores o iguales a 1, por lo que
$$
\text{cost}(x) \ge \text{manhattan}(x) \ge \sqrt{\text{manhattan}(x)}
$$

## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de _ocho puzzle_. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.


En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [14]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:

class Problema_con_Analizados(Problem):
    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda."""

    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado, accion)

In [64]:
estado_inicial = (1, 2, 3, 4, 5, 6, 7, 0, 8)
p8p = Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [65]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [66]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [67]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, puzzle_1.manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [68]:
astar_search(p8, p8.manhattan).solution()

['Mover hueco derecha']

In [69]:
astar_search(p8p, p8p.problem.manhattan).solution()

['Mover hueco derecha']

In [75]:
def resuelve_ocho_puzzle(p8, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8p = Problema_con_Analizados(p8)
    if p8.check_solvability(p8.initial):
        if h:
            sol = algoritmo(p8p, h).solution()
        else:
            sol = algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h:
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol), p8p.analizados))
    else:
        print("Este problema no tiene solucion. ")

In [154]:
p8 = Ocho_Puzzle(estado_inicial)
resuelve_ocho_puzzle(p8, astar_search, p8.manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 7 | 2 | 4 |   | 4 | 5 | 6 |   | 5 | 3 | 1 |   | H | 8 | 7 |    | 8 | H | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 3 | 6 | 1 |   | 1 | 0 | 3 |   | 8 | 0 | 4 |   | 6 | 5 | 4 |    | 1 | 2 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 8 | H | 5 |   | 7 | 8 | 2 |   | 6 | 2 | 7 |   | 3 | 2 | 1 |    | 4 | 7 | 5 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función `resuelve_ocho_puzzle`, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)

|                              | E1                   | E2                                 | E3                                  | E4                                  | E5                                |
| ---------------------------- | -------------------- | ---------------------------------- | ----------------------------------- | ----------------------------------- | --------------------------------- |
| Anchura                      | L = 0, NA = 0, T = 0 | L = 20, NA = 52506, T = 4m 5s      | L = 26, NA = 172270, T = 44m 14s    | L= 28, NA = 178035, T = 38, 31s     | L = 17, NA = 12802, T = 25.1s     |
| Profundidad                  | L = 0, NA = 0, T = 0 | L = 65020, NA = 85654, T = 42m 11s | L = 23262, NA = 24423, T = 4m 28s   | L = 65676, NA = 108205, T = 59m 7s  | L = 27205, NA = 28591, T = 5m 44s |
| Coste Uniforme               | L = 0, NA = 0, T = 0 | L = 20, NA = 48428, T = 34m 11s    | L = 26, NA = 164781, T = 3h 45m 35s | L = 28, NA = 177480, T = 4h 17m 41s | L = 17, NA = 14094, T = 1m 55s    |
| Primero el mejor (linear)    | L = 0, NA = 0, T = 0 | L = 76, NA = 748, T = 214 ms       | L = 102, NA = 777, T = 240ms        | L = 28, NA = 29, T = 1.21ms         | L = 35, NA = 139, T = 11.4ms      |
| Primero el mejor (manhattan) | L = 0, NA = 0, T = 0 | L = 140, NA = 7599, T = 16.9s      | L = 76, NA = 7608, = T = 17.6s      | L = 238, NA = 13946, T = 2m 18s     | L = 91, NA = 844, T = 228ms       |
| A* (linear)                  | L = 0, NA = 0, T = 0 | L = 20, NA = 3377, T = 3.92s       | L = 26, NA = 34703, T = 14m 24s     | L = 28, NA = 56011, T = 41m 49s     | L = 17, NA = 798, T = 215ms       |
| A* (manhattan)               | L = 0, NA = 0, T = 0 | L  20, NA = 7553, T = 18.6s        | L = 26, NA = 12975, T = 1m 50s      | L = 32, NA = 35177, T = 16m 24s     | L = 17, NA = 302, T = 36.6 ms     |
 -----------------------------------------------------------------------------------------

  Sobre E1, al no tener solución no hemos mostrado los micro segundos que tarda el programa en darse cuenta que es imposible. Por eso no lo tendremos en cuenta al analizar las diferentes formas de búsqueda. Hay que destacar que las búsquedas en Anchura, Profundidad y Coste Uniforme parecen ser menos eficientes, tanto en longitud de solución como en tiempo de búsqueda. Esto se debe a que no recurren al uso de funciones heurísticas que aproximen el coste de la solución final.
  Analicemos los resultados agrupando por filas:

- **Anchura**: como hemos estudiado en clase, la búsqueda en anchura realiza la búsqueda en el árbol de soluciones de manera que siempre estudia todos los nodos de un nivel antes de pasar al siguiente. Es por esto que las soluciones que nos presenta esta búsqueda destacan por tener una longitud(L) menor que las otras, de hecho la mínima posible. Si el árbol de estados tiene muchos hijos por cada nodo y presenta soluciones a no demasiada profundidad, entonces este algoritmo es bastante eficiente en tiempo. Si en cambio el árbol presenta soluciones a una cierta profundidad, entonces el algortimos tarda bastante en comparación con otros tipos de búsqueda.

- **Profundidad**: similar al anterior pero estudiando primero los hijos del nodo estudiado antes que sus nodos hermanos, especialmente eficiente si las soluciones se encuentran a una cierta profundidad, en comparación con la búsqueda en anchura. Con respecto a los ejemplos estudiados, es el menos eficiente en tiempo con respecto a todos ellos menos el E3 que supera al de Anchura y Profundidad, que será uno de los casos explicados anteriormente.

- **Coste uniforme**: siempre nos devuelve la solución más eficiente en longitud, pero su efiencia en tiempo y nodos estudiados es bastante pobre. Si la comparamos con las dos anteriores, dependiendo del problema podrá obtener un mejor o pero resultado, dependiendo de las características propias del problemas. En general, (igual que las dos anteriores) no tiene comparación con los resultados obtenidos con las búsquedas que presentan heurísticas.
- **Primero el mejor(linear)**: utiliza una cola de prioridad para analizar primero los nodos más prometedores, con la búsqueda linear(premia solo cuantas más cuadrículas hayan alcanzado la  casilla que les corresponde). En general es la que menos nodos necesita explorar. Es bastante eficiente tanto en tiempo como en la longitud de las soluciones obtenidad, parece más eficiente en los casos en que hay que llevar a cabo muchos movimientos complejos para alcanzar la solución ya que solo premia cuando las cuadrículas alcanzan su sitio. Es menos eficiente(con respecto a su hermana con la heurística manhattan) en el caso contrario al anterior, es decir, cuando las soluciones se consiguen con pocos movimientos pero teniendo en cuenta la cercanía de una cuadrícual a su casilla.
- **Primero el mejor(manhattan)**: muy similar al anterior  pero cambiando la heurística. Explora muchos más nodos y tiene soluciones más largas que la otra búsqueda 'Primero el mejor', pero en algunos problemas(como el E2 y el E3) obtiene mejores resultados en tiempo. En la búsqueda anterior ya explicamos por qué creemos que ocurre esto.
- **A\* linear**: este tipo de búsqueda, además de la función heurística tienen en cuenta el coste real del camino que estamos siguiendo en el árbol de soluciones. Esto provoca que consigan soluciones más cercanas a la óptima, pues si encuentra un camino mejor deshecha el que llevaba hasta ahora. También exploran más nodos que otras búsquedas por esto mismo. Son bastante eficientes en tiempo aunque dependiendo del problema pueden verse superadas por las otras búsquedas. Son las ideales si buscamos priorizar una solución casi óptima sin tener que explorar tantos nodos como la de Anchura. La búsqueda linear parece obtener siempre el camino más corto pero tarda más que su hermana con la búsqueda manhattan.
- **A\* manhattan**: explicado en la A* linear.

#### Puzzle número 1

In [155]:
e1 = Ocho_Puzzle((7,2,4,3,6,1,8,0,5))

In [156]:
%%time
resuelve_ocho_puzzle(e1, breadth_first_graph_search)

Este problema no tiene solucion. 
CPU times: user 94 µs, sys: 0 ns, total: 94 µs
Wall time: 101 µs


In [157]:
%%time
resuelve_ocho_puzzle(e1, depth_first_graph_search)

Este problema no tiene solucion. 
CPU times: user 80 µs, sys: 0 ns, total: 80 µs
Wall time: 87.5 µs


In [158]:
%%time
resuelve_ocho_puzzle(e1, uniform_cost_search)

Este problema no tiene solucion. 
CPU times: user 108 µs, sys: 0 ns, total: 108 µs
Wall time: 114 µs


In [159]:
%%time
resuelve_ocho_puzzle(e1, best_first_graph_search, e1.linear)

Este problema no tiene solucion. 
CPU times: user 81 µs, sys: 0 ns, total: 81 µs
Wall time: 87 µs


In [160]:
%%time
resuelve_ocho_puzzle(e1, best_first_graph_search, e1.manhattan)

Este problema no tiene solucion. 
CPU times: user 141 µs, sys: 0 ns, total: 141 µs
Wall time: 113 µs


In [161]:
%%time
resuelve_ocho_puzzle(e1, astar_search, e1.linear)

Este problema no tiene solucion. 
CPU times: user 71 µs, sys: 0 ns, total: 71 µs
Wall time: 76.5 µs


In [162]:
%%time
resuelve_ocho_puzzle(e1, astar_search, e1.manhattan)

Este problema no tiene solucion. 
CPU times: user 937 µs, sys: 3 µs, total: 940 µs
Wall time: 1 ms


#### Puzzle número 2

In [163]:
e2 = Ocho_Puzzle((4,5,6,1,0,3,7,8,2))

In [164]:
%%time
resuelve_ocho_puzzle(e2, breadth_first_graph_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 52506
CPU times: user 4min 5s, sys: 62.3 ms, total: 4min 5s
Wall time: 4min 5s


In [165]:
%%time
resuelve_ocho_puzzle(e2, depth_first_graph_search)

zquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda

In [166]:
%%time
resuelve_ocho_puzzle(e2, uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 20. Nodos analizados: 48428
CPU times: user 34min 10s, sys: 94.7 ms, total: 34min 11s
Wall time: 34min 11s


In [167]:
%%time
resuelve_ocho_puzzle(e2, best_first_graph_search, e2.linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [168]:
%%time
resuelve_ocho_puzzle(e2, best_first_graph_search, e2.manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mov

In [169]:
%%time
resuelve_ocho_puzzle(e2, astar_search, e2.linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3377
CPU times: user 3.92 s, sys: 0 ns, total: 3.92 s
Wall time: 3.92 s


In [170]:
%%time
resuelve_ocho_puzzle(e2, astar_search, e2.manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 7553
CPU times: user 18.6 s, sys: 0 ns, total: 18.6 s
Wall time: 18.6 s


#### Puzzle número 3

In [77]:
e3 = Ocho_Puzzle((5,3,1,8,0,4,6,2,7))

In [172]:
%%time
resuelve_ocho_puzzle(e3, breadth_first_graph_search)

Solución: ['Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 26. Nodos analizados: 172270
CPU times: user 44min 13s, sys: 276 ms, total: 44min 13s
Wall time: 44min 14s


In [78]:
%%time
resuelve_ocho_puzzle(e3, depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquie

In [173]:
%%time
resuelve_ocho_puzzle(e3, uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 26. Nodos analizados: 164781
CPU times: user 3h 45min 32s, sys: 1.18 s, total: 3h 45min 33s
Wall time: 3h 45min 35s


In [174]:
%%time
resuelve_ocho_puzzle(e3, best_first_graph_search, e3.linear)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', '

In [175]:
%%time
resuelve_ocho_puzzle(e3, best_first_graph_search, e3.manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco iz

In [176]:
%%time
resuelve_ocho_puzzle(e3, astar_search, e3.linear)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 26. Nodos analizados: 34703
CPU times: user 14min 24s, sys: 80 ms, total: 14min 24s
Wall time: 14min 24s


In [177]:
%%time
resuelve_ocho_puzzle(e3, astar_search, e3.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 26. Nodos analizados: 12975
CPU times: user 1min 50s, sys: 9.6 ms, total: 1min 50s
Wall time: 1min 50s


#### Puzzle número 4

In [142]:
e4 = Ocho_Puzzle((0,8,7,6,5,4,3,2,1))

In [76]:
%%time
resuelve_ocho_puzzle(e4, breadth_first_graph_search)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 28. Nodos analizados: 178035
CPU times: user 38min 13s, sys: 456 ms, total: 38min 13s
Wall time: 38min 31s


In [85]:
%%time
resuelve_ocho_puzzle(e4, depth_first_graph_search)

er hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco 

In [79]:
%%time
resuelve_ocho_puzzle(e4, uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 28. Nodos analizados: 177480
CPU times: user 4h 17min 38s, sys: 1.08 s, total: 4h 17min 39s
Wall time: 4h 17min 41s


In [80]:
%%time
resuelve_ocho_puzzle(e4, best_first_graph_search, e4.linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 29
CPU times: user 0 ns, sys: 1.33 ms, total: 1.33 ms
Wall time: 1.21 ms


In [81]:
%%time
resuelve_ocho_puzzle(e4, best_first_graph_search, e4.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba'

In [82]:
%%time
resuelve_ocho_puzzle(e4, astar_search, e4.linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 56011
CPU times: user 41min 49s, sys: 267 ms, total: 41min 49s
Wall time: 41min 49s


In [2]:
%%time
resuelve_ocho_puzzle(e4, astar_search, e4.manhattan)

#### Puzzle número 5

In [84]:
e5 = Ocho_Puzzle((8, 0, 3, 1, 2, 6, 4, 7, 5))

In [86]:
%%time
resuelve_ocho_puzzle(e5, breadth_first_graph_search)

Solución: ['Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 12802
CPU times: user 25.1 s, sys: 6.4 ms, total: 25.1 s
Wall time: 25.1 s


In [87]:
%%time
resuelve_ocho_puzzle(e5, depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda

In [88]:
%%time
resuelve_ocho_puzzle(e5, uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14094
CPU times: user 1min 55s, sys: 23.2 ms, total: 1min 55s
Wall time: 1min 55s


In [89]:
%%time
resuelve_ocho_puzzle(e5, best_first_graph_search, e5.linear)

Solución: ['Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 35. Nodos analizados: 139
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.4 ms


In [90]:
%%time
resuelve_ocho_puzzle(e5, best_first_graph_search, e5.manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'M

In [91]:
%%time
resuelve_ocho_puzzle(e5, astar_search, e5.linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 798
CPU times: user 217 ms, sys: 0 ns, total: 217 ms
Wall time: 215 ms


In [92]:
%%time
resuelve_ocho_puzzle(e5, astar_search, e5.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 302
CPU times: user 37.7 ms, sys: 0 ns, total: 37.7 ms
Wall time: 36.6 ms


### Ejercicio 7:  resolver usando distintas búsquedas y/o heurísticas, el problema del 15 puzzle para algunos  estados iniciales:

Escribe aquí tus conclusiones sobre los resultados y la eficiencia de las distintas opciones. 

Vamos a intentar resolver el problema de 15 puzzles con los sigueintes estados:

In [155]:
def resuelve_quince_puzzle(p15, algoritmo, h=None):
    p15p = Problema_con_Analizados(p15)
    print("Estado inicial:")
    show_15puzzle(p15.initial)
    if p15.check_solvability(p15.initial):
        if h:
            sol = algoritmo(p15p, h).solution()
        else:
            sol = algoritmo(p15p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h:
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol), p15p.analizados))
    else:
        print("Este problema no tiene solucion. ")

#### Configuración 1

In [156]:
p15_1 = Quince_Puzzle((1, 10, 15, 4, 13, 6, 3, 8, 2, 9, 12, 7, 14, 5, 0, 11))
show_15puzzle(p15_1.initial)

┌──┬──┬──┬──┐
│ 1│10│15│ 4│
├──┼──┼──┼──┤
│13│ 6│ 3│ 8│
├──┼──┼──┼──┤
│ 2│ 9│12│ 7│
├──┼──┼──┼──┤
│14│ 5│  │11│
└──┴──┴──┴──┘



In [160]:
%%time 
resuelve_quince_puzzle(p15_1, breadth_first_graph_search)
## Nota: estuvo más de 6 horas en ejecución y no terminaba. Necesitamos buscar un estado inicial tal que es fácil llegar a una solución

In [161]:
p15_2 = Quince_Puzzle((6, 13, 7, 10, 8, 9, 11, 0, 15, 2, 12, 5, 14, 3, 1, 4))
show_15puzzle(p15_2.initial)

┌──┬──┬──┬──┐
│ 6│13│ 7│10│
├──┼──┼──┼──┤
│ 8│ 9│11│  │
├──┼──┼──┼──┤
│15│ 2│12│ 5│
├──┼──┼──┼──┤
│14│ 3│ 1│ 4│
└──┴──┴──┴──┘



In [162]:
%%time 
resuelve_quince_puzzle(p15_2, breadth_first_graph_search)

In [163]:
%%time
resuelve_quince_puzzle(p15_2, depth_first_graph_search)

In [164]:
%%time
resuelve_quince_puzzle(p15_2, uniform_cost_search)

Procedemos a intentar resolver el problema del Puzzle del 15. Como este es un problema con un espacio de estados tan elevados, nos haremos valer de funciones heurísticas para guiar nuestra búsqueda.

Las funciones heurísticas que usaremos son la heurística de manhattan y la heurística lineal.

La heurística de **manhattan** asume que tenemos una fila de casillas y no un tablero, y que el coste de un estado es, para cada casilla mal colocada, la diferencia entre la posición actual de la ficha y la posición final de la ficha.

La heurística **lineal** asume que podemos intercambiar dos fichas cualesquiera en un movimiento, por lo que el coste de cada casilla con ficha distinta a la que le corresponde es 1.


#### Búsqueda expandiendo siempre el mejor estado
Hacemos una búsqueda del espacio de estados (En realidad en el espacio de búsqueda con control de repetidos) usando las heurísticas mencionadas con anterioridad

In [ ]:
%%time
resuelve_quince_puzzle(p15_2, best_first_graph_search, p15_2.linear)

In [ ]:
%%time
resuelve_quince_puzzle(p15_2, best_first_graph_search, p15_2.manhattan)

#### Búsqueda A*

Podemos combinar el coste del camino recorrido con la función heurística para realizar una búsqueda más controlada, en la que no nos fiamos ciegamente de la función heurística.

In [ ]:
%%time
resuelve_quince_puzzle(p15_2, astar_search, p15_2.linear)

In [74]:
%%time
resuelve_quince_puzzle(p15_2, astar_search, p15_2.manhattan)

###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 

Se ha hecho esto en un Jupyter Notebook aparte porque ya va lento la navegación del éste.